In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [ ]:
# passing a hashtag to get the user details

In [2]:
hashtag = 'pizzahut'
url = 'https://www.instagram.com/explore/tags/'+hashtag
browser = webdriver.Chrome(executable_path="chromedriver", service_args=["--verbose", "--log-path=chromedata.log"])
browser.get(url)
Pagelength = browser.execute_script("window.scrollTo(document.body.scrollHeight/1.5, document.body.scrollHeight/3.0);")

In [3]:
#Extract links from hashtag page
links=[]
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
page_json = script.text.split(' = ', 1)[1].rstrip(';')
data = json.loads(page_json)
## extracting shortcode of each user 
for link in data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['edges']: 
    links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')

In [10]:
## links of the loaded hashtag images to extract the details of the user
len(links)

66

In [5]:
result=pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)
        x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
        x.columns = x.columns.str.replace("shortcode_media.", "")
        result=result.append(x)
    except:
        np.nan
        
result = result.drop_duplicates(subset = 'shortcode')
result.index = range(len(result.index))

/home/aximsoft/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


In [6]:
result.columns

Index(['__typename', 'id', 'shortcode', 'dimensions.height',
       'dimensions.width', 'gating_info', 'fact_check_overall_rating',
       'fact_check_information', 'sensitivity_friction_info',
       'media_overlay_info', 'media_preview', 'display_url',
       'display_resources', 'accessibility_caption', 'is_video',
       'tracking_token', 'edge_media_to_tagged_user.edges',
       'edge_media_to_caption.edges', 'caption_is_edited',
       'has_ranked_comments', 'edge_media_to_parent_comment.count',
       'edge_media_to_parent_comment.page_info.has_next_page',
       'edge_media_to_parent_comment.page_info.end_cursor',
       'edge_media_to_parent_comment.edges',
       'edge_media_to_hoisted_comment.edges',
       'edge_media_preview_comment.count', 'edge_media_preview_comment.edges',
       'comments_disabled', 'commenting_disabled_for_viewer',
       'taken_at_timestamp', 'edge_media_preview_like.count',
       'edge_media_preview_like.edges', 'edge_media_to_sponsor_user.edges',


In [8]:
result

,__typename,id,shortcode,dimensions.height,dimensions.width,gating_info,fact_check_overall_rating,fact_check_information,sensitivity_friction_info,media_overlay_info,...,video_url,video_view_count,video_play_count,encoding_status,is_published,product_type,title,video_duration,thumbnail_src,clips_music_attribution_info
0,GraphImage,2350234661248138885,CCdtfjqgX6F,1347,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GraphSidecar,2350233741999537182,CCdtSLjBPwe,1350,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GraphVideo,2350230420279181312,CCdsh19DsAA,750,750,None,None,None,None,None,...,https://instagram.fcjb1-1.fna.fbcdn.net/v/t50....,2.0,None,None,True,feed,,60.0,https://instagram.fcjb1-1.fna.fbcdn.net/v/t51....,None
3,GraphVideo,2350228659451047344,CCdsIODha2w,750,750,None,None,None,None,None,...,https://instagram.fcjb1-1.fna.fbcdn.net/v/t50....,32.0,None,None,True,feed,,20.6,https://instagram.fcjb1-1.fna.fbcdn.net/v/t51....,None
4,GraphSidecar,2350231878295689966,CCds3D1nRru,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,GraphSidecar,2350172558822902854,CCdfX2RgoxG,1350,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,GraphImage,2350172400169854117,CCdfVihDSSl,1350,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,GraphImage,2350171795275304874,CCdfMvKhs-q,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,GraphImage,2346927773988128097,CCR9mA6D4Vh,810,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
### converting the result to a csv file
result.to_csv("pizzahut.csv")

In [9]:
### hashtag users name 
result['owner.username']

0           yooshita1392
1            ashley_aann
2           sastrisutadi
3       stellinapizzeria
4             agencia.dl
             ...        
61      ramanarestaurant
62           dellamamma_
63    diadepizzacianorte
64           foodencanta
65            ali.bari16
Name: owner.username, Length: 66, dtype: object